In [ ]:
import sys
from pathlib import Path
from os import getcwd, remove
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent.parent))

In [ ]:
import pandas as pd
import numpy as np
from os.path import join

from datetime import datetime, date, timedelta
import random


from bulletin import root, default_input, default_output, agora, hoje, ontem, anteontem, dias_apos, dias_apos_label
from bulletin.services.metabase import Metabase
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica
from bulletin.utils.static import Municipios
from bulletin.utils import utils, static
from bulletin.utils.xls_ import fit_cols
from bulletin.utils.normalize import trim_overspace

from bulletin.utils.normalize import normalize_text#normalize_hash, normalize_labels, , date_hash, normalize_number

In [ ]:
from datetime import datetime, date, timedelta

exclusao_pathfile = join(root, 'database', 'casos_confirmados')

hoje = pd.to_datetime(date.today())
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)
data_duas_semanas = hoje - timedelta(14)
print(data_duas_semanas)
data_retroativos = hoje - timedelta(31)
print(data_retroativos)

random.seed(10)

In [ ]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio']
municipios.loc[municipios['uf']!='PR','mun_resid'] = municipios.loc[municipios['uf']!='PR','municipio'] + '/' + municipios['uf']

In [ ]:
tabela_base = pd.read_excel(join(default_input, 'base_testes.xlsx'))
tabela_base

In [ ]:
cc = CasosConfirmados()
cc.load(f"cc_{hoje.strftime('%d_%m_%Y')}", compress=True)

cc.df = pd.merge(cc.df.rename(columns={'ibge_residencia':'ibge'}),municipios[['ibge','macro','rs','mun_resid','uf','municipio','regional']],on='ibge',how='left').rename(columns={'ibge':'ibge_residencia'})
cc.df = pd.merge(cc.df.rename(columns={'ibge_unidade_notifica':'ibge'}),municipios[['ibge','mun_resid']].rename(columns={'mun_resid':'mun_atend'}),on='ibge',how='left').rename(columns={'ibge':'ibge_unidade_notifica'})

In [ ]:
cc.df.columns

In [ ]:
casos_pr = cc.df.loc[cc.df['uf_residencia'] == 'PR']

In [ ]:
casos_mes = casos_pr.loc[(casos_pr['data_diagnostico'] >= data_retroativos) & (casos_pr['data_diagnostico'] <= hoje)]

In [ ]:
casos_duas_semanas = casos_pr.loc[(casos_pr['data_diagnostico'] >= data_duas_semanas) & (casos_pr['data_diagnostico'] <= hoje)]

In [ ]:
tabela_casos_mes = casos_mes.groupby('municipio')[['id_notifica']].count().reset_index().rename(columns={'id_notifica':'casos_30_dias'})
tabela_casos_mes

In [ ]:
tabela_casos_semanas = casos_duas_semanas.groupby('municipio')[['id_notifica']].count().reset_index().rename(columns={'id_notifica':'casos_14_dias'})
tabela_casos_semanas

In [ ]:
tabela = pd.merge(tabela_base, tabela_casos_mes, how='left', left_on='Municipio', right_on='municipio')
tabela = tabela.drop(columns='municipio')
tabela['casos_30_dias'] = tabela['casos_30_dias'].fillna(0).astype(int)
tabela

In [ ]:
tabela = pd.merge(tabela, tabela_casos_semanas, how='left', left_on='Municipio', right_on='municipio')
tabela = tabela.drop(columns='municipio')
tabela['casos_14_dias'] = tabela['casos_14_dias'].fillna(0).astype(int)
tabela

In [ ]:
tabela['diagnostico_diferencial'] = tabela['casos_30_dias'] * 12
tabela

In [ ]:
tabela['triagem'] = (tabela['Populacao'] / 1000) * 3 * 12
tabela['triagem'] = tabela['triagem'].astype(int)
tabela

In [ ]:
tabela['coef_incidencia_14'] = ( tabela['casos_14_dias'] /  tabela['Populacao'] ) * 100000
tabela

In [ ]:
tabela.loc[tabela['coef_incidencia_14'] >= 50, 'busca_ativa_alta'] = tabela['casos_30_dias'] *5 * 12
tabela

In [ ]:
tabela.loc[tabela['coef_incidencia_14'] < 50, 'busca_ativa_baixa'] = tabela['casos_30_dias'] *3 * 12
tabela

In [ ]:
tabela['busca_ativa_alta'] = tabela['busca_ativa_alta'].fillna(0).astype(int)
tabela['busca_ativa_baixa'] = tabela['busca_ativa_baixa'].fillna(0).astype(int)
tabela

In [ ]:
tabela.to_excel('estimativa_terceira_distribuicao.xlsx')

In [ ]:
tabela_base['RS'] = tabela_base['RS'].astype(int)
tabela_base['CAIXAS'] = tabela_base['CAIXAS'].astype(int)
tabela_base['TESTES'] = tabela_base['CAIXAS'] * testes_per_caixas
tabela_base.loc['TOTAL', ['Populacao', 'CAIXAS', 'TESTES']] = tabela_base[['Populacao', 'CAIXAS', 'TESTES']].sum()